# Demo

## Includes

In [ ]:
# mass includes
import os, sys, warnings
import ipdb
import torch as t
import numpy as np
import rawpy as rp
from tqdm.notebook import tqdm
from torchvision.utils import save_image

# add paths for all sub-folders
paths = [root for root, _, _ in os.walk('.')\
         if 'evals' not in root]
for item in paths:
    sys.path.append(item)

from ipynb.fs.full.config import Config
from ipynb.fs.full.network import *
from ipynb.fs.full.dataLoader import *
from ipynb.fs.full.util import *

## Initialization

In [ ]:
data_root = './samples'  # dataset path
save_root = './results'  # save path
denoise = True  # whether to perform denoising

In [ ]:
# for debugging only
%pdb off
warnings.filterwarnings('ignore')

# choose GPU if available
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
device = t.device('cuda' if t.cuda.is_available() else 'cpu')

# define models
net_E = Enhancer(pretrain=True).to(device)
net_E.load()
net_E.eval()

# find all folders
cam_model_list = [
    folder for folder in os.listdir(data_root)
    if os.path.isdir(os.path.join(data_root, folder))
]
cam_model_list.sort()

## Test

In [ ]:
for cam_model in cam_model_list:
    # load new pre-trained weights to denoiser
    if denoise == True:
        net_N = Denoiser(cam_model=cam_model).to(device)
        net_N.load('./saves')
        net_N.eval()

    # create a new dataloader
    test_dataset = TestSet(data_root, cam_model)
    test_loader = t.utils.data.DataLoader(test_dataset)

    # make folder for test results
    save_path = os.path.join(save_root, cam_model, 'processed')
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    # reference image
    ref_path = os.path.join(save_root, cam_model, 'original')
    if not os.path.exists(ref_path):
        os.makedirs(ref_path)

    for (noisy_raw, noise_map, cam2xyz, file) in tqdm(test_loader,
                                                      desc=cam_model,
                                                      total=len(test_loader)):
        # generate baseline
        srgb_img = demosaic(noisy_raw)
        srgb_img = cam2sRGB(srgb_img, cam2xyz)
        srgb_img = t.where(srgb_img <= 0.0031308, 12.92 * srgb_img,
                           1.055 * (srgb_img**(1 / 2.4)) - 0.055)
        save_image(srgb_img.squeeze(),
                   os.path.join(ref_path, file[0][:-4] + '.png'))

        with t.no_grad():
            # copy to device
            noisy_raw = noisy_raw.to(device)
            noise_map = noise_map.to(device)
            cam2xyz = cam2xyz.to(device)

            # initial denoising
            noisy_raw = net_N(noisy_raw, noise_map)
            srgb_img = cam2sRGB(noisy_raw, cam2xyz)

            # downsize
            down_img = downsize(srgb_img)

            # inference
            ilm_coes, clr_coes = net_E(down_img, srgb_img)
            pred_ilm_img = applyIlmDen(noisy_raw, noise_map, net_N, ilm_coes,
                                       cam2xyz)
            pred_clr_img = applyClrCoes(pred_ilm_img, clr_coes, cam2xyz)

            # save to file
            save_image(pred_clr_img.cpu().squeeze(),
                       os.path.join(save_path, file[0][:-4] + '.png'))